<a href="https://colab.research.google.com/github/abaybektursun/OS-LLMs-Workshop/blob/master/OSS_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.7/993.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.7 MB/s eta 0:00:00


In [2]:
import torch

has_gpu = False
if torch.cuda.is_available():
    install_cmd = "CMAKE_ARGS=\"-DLLAMA_CUBLAS=on\" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python"
    has_gpu = True
    print("GPU is available")
else:
    install_cmd = "CMAKE_ARGS=\"-DLLAMA_OPENBLAS=on\" FORCE_CMAKE=1 pip install llama-cpp-python"
    print("GPU is not available, using CPU")


GPU is available


In [3]:
!{install_cmd}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 42.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.57-cp310-cp310-linux_x86_64.whl size=252308 sha256=369a68b8eb6aeea3e54f7ad8f49cb3ceff20de1230c11d7ecca4e37dfa900757
  Stored in directory: /root/.cache/pip/wheels/87/9b/55/22559358a9af8074053b6c39406f5c06f0d391c052f77ca0c2
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uni

In [4]:
available_models = {
    "stable-vicuna-13B.ggmlv3.q4_0.bin":
         {
            "url":"https://huggingface.co/TheBloke/stable-vicuna-13B-GGML/resolve/main/stable-vicuna-13B.ggmlv3.q4_0.bin",
            "desc": "CarperAI StableVicuna \n" + \
                     "13 Billion Parameters, GGML, CPU Only, Q4"
         },
    "Wizard-Vicuna-13B-Uncensored.ggmlv3.q4_0.bin": 
         {
            "url":"https://huggingface.co/TheBloke/Wizard-Vicuna-13B-Uncensored-GGML/resolve/main/Wizard-Vicuna-13B-Uncensored.ggmlv3.q4_0.bin",
            "desc": "Wizard-Vicuna; 13 Billion Parameters, GGML, CPU ONLY, Q4"
         },
    "llama-7b.ggmlv3.q4_0.bin": 
         {
            "url":"https://huggingface.co/TheBloke/LLaMa-7B-GGML/resolve/main/llama-7b.ggmlv3.q4_0.bin",
            "desc": "Original LLaMa; 7 Billion Parameters, GGML, CPU/GPU, Q4"
         },
    "llama-deus-7b-v3.ggmlv3.q4_0.bin":
         {
            "url":"https://huggingface.co/TheBloke/llama-deus-7b-v3-GGML/resolve/main/llama-deus-7b-v3.ggmlv3.q4_0.bin",
            "desc": "LLaMA DEUS V3; 7 Billion Parameters, GGML, CPU/GPU, Q4"
         }
}

In [13]:
for model in available_models:
  print(model)
  print("\t" + available_models[model]['desc'])

stable-vicuna-13B.ggmlv3.q4_0.bin
	CarperAI StableVicuna 
13 Billion Parameters, GGML, CPU Only, Q4
Wizard-Vicuna-13B-Uncensored.ggmlv3.q4_0.bin
	Wizard-Vicuna; 13 Billion Parameters, GGML, CPU ONLY, Q4
llama-7b.ggmlv3.q4_0.bin
	Original LLaMa; 7 Billion Parameters, GGML, CPU/GPU, Q4
llama-deus-7b-v3.ggmlv3.q4_0.bin
	LLaMA DEUS V3; 7 Billion Parameters, GGML, CPU/GPU, Q4


In [5]:
chosen_model = "llama-7b.ggmlv3.q4_0.bin"

In [6]:
!wget {available_models[chosen_model]["url"]}

--2023-06-06 21:46:57--  https://huggingface.co/TheBloke/LLaMa-7B-GGML/resolve/main/llama-7b.ggmlv3.q4_0.bin
Resolving huggingface.co (huggingface.co)... 18.160.249.9, 18.160.249.78, 18.160.249.70, ...
Connecting to huggingface.co (huggingface.co)|18.160.249.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/b1/65/b165733c64e20fe797eaa1dca7cabbf59e1109a09e5b55b840585826f5f6098f/ec2f2d1f0dfb73b72a4cbac7fa121abbe04c37ab327125a38248f930c0f09ddf?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-7b.ggmlv3.q4_0.bin%3B+filename%3D%22llama-7b.ggmlv3.q4_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1686347217&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2IxLzY1L2IxNjU3MzNjNjRlMjBmZTc5N2VhYTFkY2E3Y2FiYmY1OWUxMTA5YTA5ZTViNTViODQwNTg1ODI2ZjVmNjA5OGYvZWMyZjJkMWYwZGZiNzNiNzJhNGNiYWM3ZmExMjFhYmJlMDRjMzdhYjMyNzEyNWEzODI0OGY5MzBjMGYwOWRkZj9yZXNwb

In [7]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [8]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [9]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

In [10]:
if has_gpu:
    llm = LlamaCpp(
        model_path=chosen_model,
        n_gpu_layers=1, n_batch=1,
        callback_manager=callback_manager, 
        verbose=True
    )
else:
    llm = LlamaCpp(
        model_path=chosen_model, 
        callback_manager=callback_manager,
        verbose=True
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [11]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
question = "What do to if I am attacked by a bear?"

llm_chain.run(question)

Llama.generate: prefix-match hit



\begin{itemize}
\item A brown bear has just attacked you on the trail, and is now trying to